## Introduction

Currently, remote monitoring devices such as watches or implants are utilized heavily to provide an enhanced healthcare for patients with periodic heart arrhythmia by constantly collecting heart activity. These devices work by collecting vast amounts of electrocardiogram (ECG) data, which is then interpreted by physicians. However, this poses an issue for physicians — there is often too much data to comb through. Thus comes the introduction of Transfer Learning for ECG classification; the goal is to utilize deep learning, specifically deep convolutional neural networks, to assist the physicians by classifying ECG data. However, such a task can be expensive due to the issue of a deep convolutional neural network requiring vast amounts of labelled data for supervised learning. We can combat this issue by breaking down the training process of the model into a few steps. First, we can pretrain the model on a general dataset of ECG data to obtain a general understanding of the data. Next, we finetune the networks on smaller, more refined, datasets to classify Atrial Fibrillation, which is the most common form of heart arrhythmia. Through the pretraining process, we can implement both supervised and unsupervised approaches to identify patterns and increase relevance of the model. 

Our goal is to implement transfer learning, in which we hope that by gathering knowledge to solve one problem, the model can apply the same knowledge to another problem within the same domain. To this end, we will pretrain CNNs on different classifications — beat, rhythm, heart rate, and future prediction. We will then finetune these models to investigate if we can accurately classify Atrial Fibrillation, a common heart arrhythmia, while reducing the number of annotations required in the dataset. 

## Scope of Reproducability

There are two main sections discussed in this paper - pretraining the models and finetuning the models. For pretraining the models, we focus on training a beat classifier, a rhythm classifier, and a heart rate classifier on the Icentia11k dataset. For the finetuning of the models, we utilize the Physionet 2017 Challenge dataset, which provides training and validation data. 

Due to the immense size of the Icentia11k dataset, there are some issues with downloading and utilizing the entire dataset for classification. For this reason, we will focus on using a subset of the dataset for pretraining our models.

## Methodology

### Data

#### Icentia11k Dataset

https://arxiv.org/abs/1910.09570

The Icentia11k dataset is the largest (as of October 21, 2019) public electrocardigram dataset. It consists of raw signals for representation learning, and is composed of such data for 11 thousand patients and 2 billion labeled beats. The goal of the dataset is to provide data foundations for semi-supervised electrocadigram models and to promote the discovery of unknown subtypes of arrhythmia and anomalous electrocadigram signal events.

For the purpose of reproducability, the entire Icentia11k dataset could not be obtained due to its size. A subset of the dataset was downloaded externally via a torrent client from the dataset's home location.

#### PhysioNet 2017 Challenge Dataset

https://physionet.org/content/challenge-2017/1.0.0/

The PhysioNet 2017 Challenge aims to encourage the development of algorithms to classify from a single short ECG lead recording (between 30-60 seconds in length) whether the recording shows normal sinus rhythm, atrial fibrillation, an alternative rhythm, or is too noisy to be classified. The types of cardiac arrhythmias that may be classified include:
- **Origin**: atrial arrhythmia, junctional arrhythmia, or ventricular arrhythmia
- **Rate**: tachycardia ( > 100 beats per minute (bpm) in adults) or bradycardia ( < 60 bpm in adults) or bradycardia ( < 60 bpm in adults)
- **Mechanism**: automaticity, re-entry, triggered
- **AV Conduction**: normal, delayed, blocked
- **Duration**: non-sustained (less than 30 s) or sustained (30 s or longer)

The dataset consists of training and validation data, partitioned separately. The training set contains 8,528 single lead ECG recordings lasting from 9 seconds to just over 60 seconds. The validation set contains 3,658 ECG recordings of similar length.

The PhysioNet 2017 Challenge Dataset was downloaded externally following instructions on the main page.

After downloading the PhysioNet 2017 Challenge Dataset, we will preprocess the data. Specifically, we will normalize the dataset, adapt the sampling frequency to be similar to the Icentia11k, zero-pad the recordings, and finally split the dataset into train and test sets.

In [6]:
# Finetuning code
# Finetuning is done on Physionet2017 dataset, here we are preprocessing the data

from finetuning import datasets
from finetuning.utils import train_test_split
from transplant.utils import save_pkl
data = datasets.get_challenge17_data(
   db_dir='data/physionet/files/challenge-2017/1.0.0/training',
   fs=250,  # keep sampling frequency the same as Icentia11k
   pad=16384,  # zero-pad recordings to keep the same length at about 65 seconds
   normalize=True)  # normalize each recording with mean and std computed over the entire dataset
# maintain class ratio across both train and test sets by using the `stratify` argument
train_set, test_set = train_test_split(
   data, test_size=0.2, stratify=data['y'])
save_pkl('data/physionet_train.pkl', **train_set)
save_pkl('data/physionet_test.pkl', **test_set)

data/physionet/files/challenge-2017/1.0.0/training


### Model

There are two types of models - pretraining and finetuning. For each type of model, there are three main models for pretraining: beat classification, rhythm classification, and heart rate classification.

#### Pretraining Models

All the pretraining models are trained on the Icentia11k dataset.

- **Beat Classification**: The beat classification model's goal is to classify individual heartbeats into different categories, such as normal beats, premature ventricular contractions (PVCs), and atrial premature beats (APBs), to name a few. The model is trained on the Icentia11k dataset, which includes labeled ECG segments with annotations indicating the type of beat. The model architecture can vary between ResNet18, ResNet34, or others -- specifically, we have used ResNet18 for reproducability -- which is finetuned with the Icentia11k dataset for the task of beat classification. The final layer of the model is a softmax activation to output probability scores for each beat class.

- **Rhythm Classification**: The rhythm classification pretrained model is designed for the task of rhythm classification, which involves categorizing longer segments of ECG data into different rhythm classes, such as normal sinus rhythm, atrial fibrillation, sinus tachycardia, to name a few. It is trained on the Icentia11k dataset with annotations indicating the rhythm type for each segment. Similar to the beat classification pretrained model, the architecture is adapted from ResNet18 and finetuned for rhythm classification, with modifications to handle longer input sequences. Finally, the output layer of the model consists of a softmax activation to predict probability scores over each rhythm class. 

- **Heart Rate Classification**: The heart rate classification pretrained model is designed for the task of heart rate classification, which involves classifying ECG segments based on their heart rate characteristics, with labels such as normal heart rate, bradycardia, and more. It is also trained on the Icentia11k dataset, with labeled annotations corresponding to heart rates. Similar to the other pretrained models, the model architecture of the heart rate classification model is also an adaptation of ResNet18, finetuned with the Icentia11k dataset for the heart rate classification task. Similar to the other models, it also incorporates a final layer of softmax activation to predict probability scores over each heart rate class. 

#### Finetuning Models

All the finetuning models are trained on the PhysioNet 2017 Challenge Dataset.

- **Beat Classification**: The pretrained beat classification model is fine-tuned on the PhysioNet 2017 Challenge Dataset to become more specific. This process involves modifying the best weights from the pretrained beat classification model to adapt to nuances and characteristics of the new dataset.

- **Rhythm Classification**: The pretrained rhythm classification model is fine-tuned on the PhysioNet 2017 Challenge Dataset to focus on the rhythm domain in the target dataset. The goal of the fine-tuning is to update the model parameters and make it more adept at distinguishing between the various rhythm classes in the PhysioNet 2017 Challenge Dataset, with a goal of better classification.

- **Heart Rate Classification**: The pretrained heart rate classification model is fine-tuned on the PhysioNet 2017 Challenge Dataset to fine-tune for the target dataset. The model parameters are adjusted to allow the model to adapt its representations to the specific heart rate patterns present in the PhysioNet 2017 Challenge Dataset, which may vary from patterns seen in the pretrained dataset.

### Training

To train the model on the full dataset, we would require more storage to host the entire Icentia11k dataset. 

#### Pretraining Models

Below, we have the pretraining code to run functions that create the beat classification pretrained model, rhythm classification pretrained model, and heart rate classification pretrained model. 

For simplicity, we have separated all of the data preprocessing and training code to separate python files, and utilize scripts to perform the task at hand. 

For demonstration sake, we will utilize 2 epochs.

Note that because we only have a subset of the dataset, there will be multiple lines indicating that files are missing for specific patient ids. For purpose of reproducability, we will skip these patients.

In [10]:
# Pretrainer code
# Pretrains the beat classification task on the Icentia11k dataset for 10 epochs
!/opt/anaconda3/bin/python -m pretraining.trainer \
--job-dir "jobs/beat_classification" \
--task "beat" \
--train "data/icentia11k" \
--arch "resnet18" \
--epochs 2

Creating working directory in jobs/beat_classification
Setting random state 50683
Building train data generators
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
Building model ...
# model parameters: 4,495,045
Epoch 1/2
2024-04-14 14:16:33.000292: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-04-14 14:16:47.292620: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 12001 of 16000
2024-04-14 14:16:49.924907: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.

Epoch 1: saving model to jobs/beat_classification/epoch_01/model.weights.h5
100/100 - 74s - 739ms/step - acc: 0.7675 - loss: 0.7381
Epoch 2/2

Epoch 2: saving model to jobs/beat_classification/epoch_02

In [13]:
# Extracts best checkpoint for beat classification
from pretraining.utils import get_pretrained_weights
resnet18 = get_pretrained_weights(
   checkpoint_file='jobs/beat_classification/epoch_02/model.weights.h5',
   task='beat',
   arch='resnet18')
resnet18.save_weights('jobs/beat_classification/resnet18.weights.h5')

In [11]:
# Pretrainer code
# Pretrains the beat classification task on the Icentia11k dataset for 10 epochs
!/opt/anaconda3/bin/python -m pretraining.trainer \
--job-dir "jobs/rhythm_classification" \
--task "rhythm" \
--train "data/icentia11k" \
--arch "resnet18" \
--epochs 2

Creating working directory in jobs/rhythm_classification
Setting random state 32027
Building train data generators
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
Building model ...
# model parameters: 4,495,558
Epoch 1/2
2024-04-14 14:18:46.657768: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-04-14 14:19:00.563738: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 11446 of 16000
2024-04-14 14:19:03.899534: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.

Epoch 1: saving model to jobs/rhythm_classification/epoch_01/model.weights.h5
100/100 - 68s - 677ms/step - acc: 0.8169 - loss: 0.5786
Epoch 2/2

Epoch 2: saving model to jobs/rhythm_classification/ep

In [17]:
# Extracts best checkpoint for rhythm classification
from pretraining.utils import get_pretrained_weights
resnet18 = get_pretrained_weights(
   checkpoint_file='jobs/rhythm_classification/epoch_02/model.weights.h5',
   task='rhythm',
   arch='resnet18')
resnet18.save_weights('jobs/rhythm_classification/resnet18.weights.h5')

In [12]:
# Pretrainer code
# Pretrains the beat classification task on the Icentia11k dataset for 10 epochs
!/opt/anaconda3/bin/python -m pretraining.trainer \
--job-dir "jobs/hr_classification" \
--task "hr" \
--train "data/icentia11k" \
--arch "resnet18" \
--epochs 2

Creating working directory in jobs/hr_classification
Setting random state 65113
Building train data generators
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
Building model ...
# model parameters: 4,494,532
Epoch 1/2
2024-04-14 14:20:44.511507: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-04-14 14:20:58.425637: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 12001 of 16000
2024-04-14 14:21:00.865455: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.

Epoch 1: saving model to jobs/hr_classification/epoch_01/model.weights.h5
100/100 - 69s - 695ms/step - acc: 0.7694 - loss: 0.6462
Epoch 2/2

Epoch 2: saving model to jobs/hr_classification/epoch_02/model

In [18]:
# Extracts best checkpoint for beat classification
from pretraining.utils import get_pretrained_weights
resnet18 = get_pretrained_weights(
   checkpoint_file='jobs/hr_classification/epoch_02/model.weights.h5',
   task='hr',
   arch='resnet18')
resnet18.save_weights('jobs/hr_classification/resnet18.weights.h5')

#### Finetuning Models

Below, we have the finetuning code to run functions that create the beat classification pretrained model, rhythm classification pretrained model, and heart rate classification pretrained model. 

For simplicity, we have separated all of the data preprocessing and training code to separate python files, and utilize scripts to perform the task at hand. 

In the finetuning, we will use the resnet18 weights from each specific pretrained model that we determined to be the best and finetune the model using those pretrained weights.

**--Currently debugging--**

In [21]:
# !/opt/anaconda3/bin/python -m finetuning.trainer \
# --job-dir "jobs/af_classification" \
# --train "data/physionet_train.pkl" \
# --test "data/physionet_test.pkl" \
# --weights-file "jobs/beat_classification/resnet18.weights.h5" \
# --val-size 0.0625 \
# --arch "resnet18" \
# --batch-size 64 \
# --epochs 2

### Evaluation

We can analyze the metrics from the epochs that are saved to each tasks 'history.csv' file.

In [23]:
import csv
# Open the CSV file
with open('jobs/beat_classification/history.csv', 'r') as f:
    # Create a CSV reader object
    reader = csv.reader(f)
    # Iterate over the rows in the CSV file
    for row in reader:
        # Print the row
        print(row)

['epoch', 'acc', 'loss', 'val_acc', 'val_loss']
['0', '0.7674999833106995', '0.7380645871162415', 'NA', 'NA']
['1', '0.8159375190734863', '0.537787675857544', 'NA', 'NA']


In [24]:
import csv
# Open the CSV file
with open('jobs/beat_classification/history.csv', 'r') as f:
    # Create a CSV reader object
    reader = csv.reader(f)
    # Iterate over the rows in the CSV file
    for row in reader:
        # Print the row
        print(row)

['epoch', 'acc', 'loss', 'val_acc', 'val_loss']
['0', '0.7674999833106995', '0.7380645871162415', 'NA', 'NA']
['1', '0.8159375190734863', '0.537787675857544', 'NA', 'NA']


In [25]:
import csv
# Open the CSV file
with open('jobs/beat_classification/history.csv', 'r') as f:
    # Create a CSV reader object
    reader = csv.reader(f)
    # Iterate over the rows in the CSV file
    for row in reader:
        # Print the row
        print(row)

['epoch', 'acc', 'loss', 'val_acc', 'val_loss']
['0', '0.7674999833106995', '0.7380645871162415', 'NA', 'NA']
['1', '0.8159375190734863', '0.537787675857544', 'NA', 'NA']


## Results

### Results

From the results of the evaluation of the pretrained model, we can note that the pretrained models already have moderately high accuracies, indicating that the fine-tuned model should perform much better at the main task at hand. 

### Analyses

Todo

### Plans

For future plans, we can try to use more current ResNet models to see if we can improve model accuracy in any way.
For the future prediction model, we can remove the attention pooling module to test if we can still extract important similarities between the context frames and the future frames. 
